# Guía de Instalación y Práctica — Sesión 1 (Notebook)
Curso: **IA en el Aula — Nivel Avanzado**  
Profesor: **Luis Daniel Benavides Navarro**  
Fecha: **22 de octubre de 2025**

Este cuaderno guía a los participantes para configurar el entorno, conectarse a una API de modelos de lenguaje y realizar las primeras consultas con **parámetros clave** como `temperature`, `max_tokens`, `top_p`, etc. Se incluyen explicaciones paso a paso y ejercicios.


## 1) Requisitos previos
- Python 3.10 o superior
- Cuenta y **clave API** en un proveedor de modelos de lenguaje (ej. OpenAI)
- Editor/entorno: VSCode, Jupyter o Colab
- Conexión a Internet

Si estás en **Colab**, puedes ejecutar el siguiente comando para instalar dependencias. En local, usa tu terminal.

In [1]:
# (Opcional en local/Colab) Instalar dependencias
%pip install --quiet openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


## 2) Preparar variables de entorno
Crea un archivo `.env` en la carpeta del proyecto con:

```
OPENAI_API_KEY=tu_clave_aqui
```
Nunca publiques tu clave. Evita subir `.env` a repositorios públicos.


In [4]:
# 3) Cargar la clave y crear el cliente
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # Lee el archivo .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("Cliente inicializado. Modelo listo para consultas.")

Cliente inicializado. Modelo listo para consultas.


Si está en Google Colab, debe incluir la llave en los secretos (en el menú de la izquierda), activar la variable y luego inicializar así:

In [3]:
# 3) Cargar la clave y crear el cliente en google Colab
from openai import OpenAI
from google.colab import userdata


client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
print("Cliente inicializado. Modelo listo para consultas.")

ModuleNotFoundError: No module named 'google'

## 4) Parámetros importantes de la API (explicación breve)
- **`model`**: identifica el modelo a consultar (p.ej., `gpt-4o-mini`).
- **`messages`**: lista de turnos conversacionales. Usa roles `system` (instrucciones generales), `user` (tu prompt), `assistant` (respuestas previas si las hay).
- **`temperature`**: *aleatoriedad* de la salida. Valores bajos (0.0–0.3) hacen respuestas más deterministas; valores altos (0.7–1.0) hacen respuestas más creativas.
- **`top_p`**: alternativa a `temperature` basada en muestreo por probabilidad acumulada; usa uno u otro (no ambos a la vez a valores lejanos) para afinar el estilo.
- **`max_tokens`**: tope de tokens generados en la respuesta. Si es muy bajo, el texto puede cortarse.
- **`stop`**: secuencias de corte; si aparecen, la generación se detiene.
- **`frequency_penalty` / `presence_penalty`**: penalizaciones para reducir repeticiones y fomentar aparición de nuevos términos.

**Buenas prácticas:**
- Controlar `temperature` (0.2–0.7) según la tarea (evaluación → bajo; lluvia de ideas → medio/alto).
- Estructurar prompts y, cuando sea útil, **pedir JSON** para facilitar la automatización.
- Evitar incluir datos personales o sensibles en prompts.


In [5]:
# 5) Primera consulta: respuesta libre
prompt = "Explica en dos frases qué es la inteligencia artificial en la educación."
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7  # más creativo que 0.2, menos que 1.0
)
print(response.choices[0].message.content)

La inteligencia artificial en la educación se refiere al uso de tecnologías avanzadas para personalizar el aprendizaje, adaptando contenido y métodos a las necesidades individuales de los estudiantes. Además, puede facilitar la automatización de tareas administrativas y proporcionar análisis de datos para mejorar la enseñanza y el rendimiento académico.


### Comentarios sobre `temperature`
- A `0.0–0.2`: respuestas casi siempre iguales (útil en **rubricas** o **explicaciones estándar**).
- A `0.5–0.8`: más variación léxica/estilística (útil en **generación de materiales** o **brainstorming**).
- A `>0.9`: creatividad alta pero riesgo de salidas menos precisas.


In [7]:
# 6) Respuesta estructurada en JSON para automatización
import json

query = "¿Qué es aprendizaje supervisado?"
schema_instruction = (
    "Responde en formato JSON con las claves: operation, input, output. "
    "operation debe ser 'explanation'; input debe repetir la pregunta; output la explicación clara y breve."
)
response_json = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": schema_instruction},
        {"role": "user", "content": query}
    ],
    temperature=0.3,       # más determinista para formatos estructurados
    max_tokens=300         # suficiente para una explicación breve
)
text = response_json.choices[0].message.content
print(text)

# (Opcional) intentar cargar como JSON si el modelo devolvió un objeto válido
try:
    data = json.loads(text)
    print("\nValid JSON →", data)
except json.JSONDecodeError:
    print("\nLa salida no es JSON válido literal. Puedes parsearla manualmente o usar validadores/funciones JSON del proveedor.")

{
  "operation": "explanation",
  "input": "¿Qué es aprendizaje supervisado?",
  "output": "El aprendizaje supervisado es un tipo de aprendizaje automático en el que un modelo es entrenado utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida correspondiente conocida, lo que permite al modelo aprender a predecir la salida a partir de nuevas entradas. Se utiliza comúnmente en tareas como clasificación y regresión."
}

Valid JSON → {'operation': 'explanation', 'input': '¿Qué es aprendizaje supervisado?', 'output': 'El aprendizaje supervisado es un tipo de aprendizaje automático en el que un modelo es entrenado utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida correspondiente conocida, lo que permite al modelo aprender a predecir la salida a partir de nuevas entradas. Se utiliza comúnmente en tareas como clasificación y regresión.'}


## 7) Ejercicios propuestos
1. Cambia `temperature` a 0.1, 0.5 y 0.9 y compara el estilo de las respuestas.
2. Pide que el modelo responda **siempre en JSON** usando un `system` que lo exija. Verifica si cumple.
3. Crea un prompt de tu área (p.ej., *programación*, *arquitectura*, *matemáticas*) que devuelva un JSON con `operation`, `input`, `steps` (lista) y `output`.
4. Limita la longitud con `max_tokens` y observa si corta.


In [8]:
# Plantilla reutilizable para el curso
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.3,
              max_tokens: int = 100,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "prompt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de predicción que utiliza un enfoque de ramificación para tomar decisiones basadas en características de los datos. Cada nodo interno representa una prueba sobre un atributo, cada rama representa el resultado de esa prueba y cada hoja representa una clase o resultado final, permitiendo clasificar o predecir resultados a partir de las decisiones tomadas en cada nodo."



## 9) Solución de problemas comunes
- **`openai.AuthenticationError` / `401`**: clave inválida o no cargada; revisa tu `.env` y reinicia el kernel.
- **`Rate limit`**: excediste el número de solicitudes por minuto; espera unos segundos y reintenta.
- **`model_not_found`**: el modelo no existe o no tienes acceso; cambia a uno disponible en tu cuenta.
- **Salidas no JSON**: fija `temperature=0.0–0.3`, agrega instrucciones `system` estrictas y/o usa funciones nativas de validación JSON si el proveedor las ofrece.


## 10) Próximos pasos
En la siguiente sesión se verán **consultas avanzadas**, manejo de contexto y el punto de partida para construir un **asistente con RAG** (recuperación de conocimiento) usando materiales del curso.
